# 전남A2/졸지아
**프로젝트 개요:** bard api를 이용한 뉴스 긍정 분석 기반 주식 자동 매수, 매도 프로그램\
\
**목적:** 신속한 최신 정보 취득이 어려운 투자자들을 돕기 위함이다.\
\
**기술 활용 방식:** 코드와 설명서를 배포하거나 서비스 자체를 웹을 만든다.\
\
**결과:** 원하는 종목이 실시간 상황에 따라 자동으로 매수, 매도된다.\
\
**기술 기대효과:** 빠른 정보 획득이 어려운 투자자들이 수익을 얻을 수 있다.\
\
**팀원 역할:**

**이서율:** bard api 이용 및 코드 합쳐서 연결하기\
**정인강:** 뉴스 크롤링 방식 연구, 텍스트 분석\
**박도현:** 한국투자증권 api를 이용해 자동 매수, 매도 프로그램 제작

___



# module, key값들

In [34]:
!pip install bardapi
!pip install mojito2

from bardapi import Bard
import mojito
import pprint
import time
from time import sleep

token ='cAh07OSiW0A-ZYmNCBYvjGWbkrkBHWDiubGfFUsQIgQZe9g_qpvtVntDelTPNbj39-BYaA.'

key = "PSIQ599g3fqtNUhwzRRJwH1oJGWSTpftdYVM"
secret = "RZkYtlntKFzVRN7678QBwZCk01Img7WgaTP1cNOciyGxAbBDj7z/5y4+v9jZoHrLmkO/bMWCB7+lIwTj5gfgIQlmb5jRsXWIEJvSuDDUaBsiT/PLRLxm6vxGVG58rxij21wLu5I6LkvwW+XhzLCvvbCdNcbHuxdwMRbMTosaI963J7imvaI="
ACC_NO = "50096816-01"

broker = mojito.KoreaInvestment(api_key=key, api_secret=secret, acc_no=ACC_NO, mock=True)
resp2 = broker.fetch_balance()

___________

# 잔고 조회

In [41]:
tot_evlu_amt = int(resp2['output2'][0]['tot_evlu_amt']) #nass_amt 순자산 금액, tot_evlu_amt 총평가 금액
nass_amt = int(resp2['output2'][0]['nass_amt'])
print('___________________')
print('총평가금액: %d원' %tot_evlu_amt)
print('순자산금액: %d원' %nass_amt)

___________________
총평가금액: 10007927원
순자산금액: 10007927원


___________

# 국내 주식

In [ ]:
broker = mojito.KoreaInvestment(api_key=key, api_secret=secret, acc_no=ACC_NO, mock=True)
prev_inza=None

while True:
    #답변 실시간성 확인 성공!
    banswer=Bard(token=token).get_answer("이스라엘-하마스에 대한 가장 최신 뉴스 1개를 근거로 전쟁이 장기화될 것 같으면 '장,장,장' 이라고 말해줘, 전쟁이 금방 끝날 것 같으면 '끝,끝,끝' 이라고 말해줘")['content']

    #텍스트 개수 파악
    buy_count = banswer.count('장')
    sell_count = banswer.count('끝')

    #비교
    if buy_count > sell_count:
        inza = 1
    else:
        inza = 0

    print(banswer)
    print('_______________________\n인자 값:%d\n' %inza)

    if inza == prev_inza:
        print("현상유지\n")

    else:
      if inza == 1:
        print("매수하겠습니다.\n")
        resp = broker.create_market_buy_order(symbol="024060", quantity=10)
      elif inza == 0:
        print("매도하겠습니다.\n")
        resp = broker.create_market_sell_order(symbol="024060", quantity=10)

      pprint.pprint(resp)

      prev_inza = inza


    print("\n이 코드는 5분마다 반복됩니다.\n_______________________")

    time.sleep(300)

_________

# 미국 주식

In [ ]:
broker = mojito.KoreaInvestment(api_key=key, api_secret=secret, acc_no=ACC_NO, exchange='나스닥',mock=True)
prev_inza=None

while True:
    #답변 실시간성 확인 성공!
    banswer=Bard(token=token).get_answer("이스라엘-하마스에 대한 가장 최신 뉴스 1개를 근거로 전쟁이 장기화될 것 같으면 '장,장,장' 이라고 말해줘, 전쟁이 금방 끝날 것 같으면 '끝,끝,끝' 이라고 말해줘")['content']

    #텍스트 개수 파악
    buy_count = banswer.count('장')
    sell_count = banswer.count('끝')

    #비교
    if buy_count > sell_count:
        inza = 1
    else:
        inza = 0

    print(banswer)
    print('\n______________________\n인자 값:%d\n' %inza)

    if inza == prev_inza:
        print("현상유지\n")

    else:
      if inza == 1:
        print("매수하겠습니다.\n")
        pricek = float(broker.fetch_price("AAPL")['output']['last'])
        resp = broker.create_limit_buy_order(symbol="AAPL", price=pricek,quantity=1)

      elif inza == 0:
        print("매도하겠습니다.\n")
        pricek = float(broker.fetch_price("AAPL")['output']['last'])
        resp = broker.create_limit_sell_order(symbol="AAPL", price=pricek,quantity=1)

      pprint.pprint(resp)
      prev_inza=inza

    print("\n이 코드는 5분마다 반복됩니다.\n_______________________")

    time.sleep(300)